In [2]:
import tensorflow as tf
import numpy as np

In [3]:
# misc utils:

#1. tokenize using google sentence-piece split vocab 


def load_vocab(vocab_fpath):
    vocab = [line.split()[0] for line in open(vocab_fpath,'r').read().splitlines()]
    token2idx = {token:idx for idx,token in  enumerate(vocab)}
    idx2token = {idx:token for idx,token in enumerate(vocab)}
    return token2idx,idx2token

# load length controlled sentences

def load_data(fpath1,fpath2,maxlen1,maxlen2):
    sents1,sents2 = [],[]
    with open(fpath1,'r') as f1, open(fpath2,'r') as f2:
        for sent1,sent2 in zip(f1,f2):
            if len(sent1.split())+1 > maxlen1:
                continue
            if eln(sent2.split())+1 > maxlen2:
                continue
            sents1.append(sent1.strip())
            sents2.append(sent2.strip())
    return sents1,sents2




In [ ]:
# Now coding the batching and iterator functions

def sent2num_encode(inp_sent,type_inp,t2idx_dict):
    inp_str = inp_sent.decode("utf-8")
    if type_inp == "x":
        tokens = inp_str.split() + ["</s>"]
    else:
        tokens= ["<s>"] inp_str.split() + ["</s>"]
        
    enc_list = [t2idx_dict.get(t,t2idx_dict["<unk>"]) for t in tokens]
    return enc_list


def gen_fn(sents1,sents2,vocab_fpath):
    token2idx, _ = load_vocab(vocab_fpath) 
    #Now the generator part using yield
    for sent1,sent2 in zip(sents1,sents2):
        x = sent2num_encode(sents1,"x",token2idx)
        y = sent2num_encode(sents2,"y",token2idx)
        
        decoder_inp,y = y[:-1], y[1:]
        x_seq_len, y_seq_len =  len(x),len(y)
        yield (x,x_seq_len,sent1) , (decoder_inp,y,y_seq_len,sent2)


def data_feed_fn(sents1,sents2,vocab_fpath,batch_size,shuffle=False):
    # xs = (x[int32] , x_seq_len[int32], sents1[string])
    # ys = (decoder_inp[int32], y[int32], y_seq_len[int32], sents2[string])
    shapes=(([None],(), ()),([None],[None],(),()))  # represents xs, ys
    
    types = (
    (tf.int32,tf.int32,tf.string),
    (tf.int32,tf.int32,tf.int32,tf.string)
    )
    
    paddings = (
    (0,0,''),
    (0,0,0,'')
    )
    

def get_batch(fpath1,fpath2,maxlen1,maxlen2,vocab_fpath,batch_size,shuffle=False):
    sents1,sents2 = load_data(fpath1,fpath2,maxlen1,maxlen2)
    
